# 🧠 Sentiment Analysis on Amazon Reviews (PySpark + PyTorch)

Mục tiêu: 
- Làm sạch dữ liệu đánh giá sản phẩm bằng PySpark.
- Chuyển đổi dữ liệu đã xử lý sang tensor để huấn luyện mô hình PyTorch.

Công cụ:
- PySpark (xử lý dữ liệu lớn)
- PyTorch (huấn luyện mô hình Deep Learning)


## Khám phá dữ liệu (Exploratory Data Analysis - EDA)

Trong phần này, chúng ta sẽ:
- Đọc dữ liệu gốc từ file `.csv`
- Kiểm tra schema, số lượng dòng/cột
- Kiểm tra dữ liệu thiếu (null/missing)
- Chuẩn hoá cột `Rating`
- Phân tích phân bố điểm đánh giá và cảm xúc
- Tính toán độ dài review
- Phân tích theo thời gian (nếu có cột ngày đánh giá)


### Import thư viện

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import Tokenizer, StopWordsRemover

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

### Tạo Spark Session

In [21]:
# ==============================================================
# 🔹 Tạo SparkSession
# ==============================================================

spark = SparkSession.builder \
    .appName("Amazon_Review_Sentiment") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

### Đọc dữ liệu gốc

In [22]:
# ==============================================================
# 🔹 Đọc dữ liệu gốc
# ==============================================================

data_path = "../data/raw/Amazon_Reviews.csv"

df = spark.read.option("header", True).option("inferSchema", True).csv(data_path)

print("Đọc dữ liệu thành công.")
df.show(5, truncate=False)


Đọc dữ liệu thành công.
+----------------+-------------------------------+-------+------------+------------------------+----------------------+-----------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+
|Reviewer Name   |Profile Link                   |Country|Review Count|Review Date             |Rating                |Review Title                                   |Review Tex

### Thông tin tổng quan về dữ liệu

In [23]:
# ==============================================================
# 🔹 Thông tin tổng quan
# ==============================================================

print("Schema:")
df.printSchema()

print(f"Tổng số dòng: {df.count():,}")
print(f"Tổng số cột: {len(df.columns)}")
print("Danh sách cột:", df.columns)


Schema:
root
 |-- Reviewer Name: string (nullable = true)
 |-- Profile Link: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Review Count: string (nullable = true)
 |-- Review Date: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Review Title: string (nullable = true)
 |-- Review Text: string (nullable = true)
 |-- Date of Experience: string (nullable = true)

Tổng số dòng: 21,656
Tổng số cột: 9
Danh sách cột: ['Reviewer Name', 'Profile Link', 'Country', 'Review Count', 'Review Date', 'Rating', 'Review Title', 'Review Text', 'Date of Experience']


### Kiểm tra giá trị thiếu

In [24]:
# ==============================================================
# 🔹 Kiểm tra giá trị thiếu
# ==============================================================

null_df = df.select([
    F.count(F.when(F.col(c).isNull() | (F.col(c) == ''), c)).alias(c)
    for c in df.columns
])
print("Số lượng giá trị null hoặc rỗng trong từng cột:")
null_df.show(vertical=True)

Số lượng giá trị null hoặc rỗng trong từng cột:
-RECORD 0-----------------
 Reviewer Name      | 0   
 Profile Link       | 141 
 Country            | 436 
 Review Count       | 523 
 Review Date        | 565 
 Rating             | 581 
 Review Title       | 590 
 Review Text        | 595 
 Date of Experience | 920 



### Thống kê cột Rating

In [25]:
# ==============================================================
# 🔹 Thống kê cột Rating
# ==============================================================

df = df.withColumn(
    "Rating",
    F.regexp_extract(F.col("Rating"), r"(\d)(?=\s*out of 5)", 1).cast(IntegerType())
)

# Thống kê điểm đánh giá
rating_stats = df.groupBy("Rating").agg(
    F.count("*").alias("count")
).orderBy("Rating")

print("\n Thống kê điểm Rating:")
rating_stats.show()


 Thống kê điểm Rating:
+------+-----+
|Rating|count|
+------+-----+
|  NULL|  601|
|     1|13123|
|     2| 1227|
|     3|  885|
|     4| 1292|
|     5| 4528|
+------+-----+



### Thống kê độ dài review

In [26]:
# ==============================================================
# 🔹 Thống kê độ dài nội dung Review
# ==============================================================

if "Review Text" in df.columns:
    df = df.withColumn("ReviewLength", F.length(F.col("Review Text")))

    review_stats = df.select(
        F.mean("ReviewLength").alias("mean_len"),
        F.min("ReviewLength").alias("min_len"),
        F.max("ReviewLength").alias("max_len")
    ).collect()[0]

    print(f"Độ dài trung bình: {review_stats['mean_len']:.2f}")
    print(f"Ngắn nhất: {review_stats['min_len']}, Dài nhất: {review_stats['max_len']}")


Độ dài trung bình: 422.60
Ngắn nhất: 6, Dài nhất: 8166


### Phân tích theo thời gian

In [27]:
# ==============================================================
# 🔹 Phân tích theo thời gian (nếu có cột Review Date)
# ==============================================================

if "Review Date" in df.columns:
    df = df.withColumn("Review Date", F.to_date(F.col("Review Date")))
    time_dist = df.groupBy(F.year("Review Date").alias("year")).count().orderBy("year")
    print("Số lượng review theo năm:")
    time_dist.show()

Số lượng review theo năm:
+----+-----+
|year|count|
+----+-----+
|NULL|  601|
|2007|    1|
|2008|    3|
|2009|    9|
|2010|   22|
|2011|  360|
|2012| 1166|
|2013|   88|
|2014|  166|
|2015|  175|
|2016|  317|
|2017|  616|
|2018| 1041|
|2019| 2314|
|2020| 2638|
|2021| 2750|
|2022| 2551|
|2023| 4017|
|2024| 2821|
+----+-----+



## Tiền xử lý dữ liệu (Data Preprocessing)

Trong phần này, chúng ta sẽ:
- Chuẩn hoá cột **Rating** về giá trị số (1–5)
- Gán nhãn **Sentiment** (`positive`, `negative`)
- Làm sạch văn bản: xoá ký tự đặc biệt, chữ thường, loại bỏ stopwords
- Kết hợp tiêu đề + nội dung review
- Lưu dữ liệu đã chuẩn hoá ra file `.parquet` để phục vụ huấn luyện mô hình.


### Chuẩn hoá cột Rating

In [28]:
# ==============================================================
# 🔹 Chuẩn hoá cột Rating
# ==============================================================
df = df.withColumn(
    "Rating",
    F.regexp_extract(F.col("Rating"), r"(\d+)", 1).cast(IntegerType())
)

df = df.filter((F.col("Rating") >= 1) & (F.col("Rating") <= 5))
print("Đã chuẩn hoá cột Rating về khoảng [1–5].")

df.select("Rating").distinct().orderBy("Rating").show()

Đã chuẩn hoá cột Rating về khoảng [1–5].
+------+
|Rating|
+------+
|     1|
|     2|
|     3|
|     4|
|     5|
+------+



### Gán nhãn Sentiment

In [29]:
# ==============================================================
# 🔹 Gán nhãn cảm xúc dựa trên Rating
# ==============================================================

df = df.withColumn(
    "Sentiment",
    F.when(F.col("Rating") <= 2, "negative")
     .otherwise("positive")
)

df.groupBy("Sentiment").count().show()


+---------+-----+
|Sentiment|count|
+---------+-----+
| positive| 6705|
| negative|14350|
+---------+-----+



### Làm sạch văn bản

In [30]:
# ==============================================================
# 🔹 Làm sạch nội dung Review
# ==============================================================

if "Review Title" in df.columns and "Review Text" in df.columns:
    df = df.withColumn("FullReview", F.concat_ws(". ", "Review Title", "Review Text"))
elif "Review Text" in df.columns:
    df = df.withColumnRenamed("Review Text", "FullReview")

df = (
    df.withColumn("FullReview", F.lower("FullReview"))
      .withColumn("FullReview", F.regexp_replace("FullReview", r"[^a-zA-Z0-9\s]", " "))
      .withColumn("FullReview", F.regexp_replace("FullReview", r"\s+", " "))
      .filter(F.length("FullReview") > 10)
)

print("Đã làm sạch văn bản.")
df.select("FullReview").show(5, truncate=False)


Đã làm sạch văn bản.
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|FullReview                                                                                                                                                                                                                                                                                                                                              

### Loại bỏ Stopwords

In [31]:
# ==============================================================
# 🔹 Tokenization & Stopword Removal
# ==============================================================

tokenizer = Tokenizer(inputCol="FullReview", outputCol="Words")
remover = StopWordsRemover(inputCol="Words", outputCol="FilteredWords")

df = tokenizer.transform(df)
df = remover.transform(df)
df = df.withColumn("CleanText", F.concat_ws(" ", "FilteredWords"))
df = df.filter(F.length("CleanText") > 10)

df.select("CleanText").show(5, truncate=False)
print("Đã loại bỏ stopwords.")


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CleanText                                                                                                                                                                                                                                                                                                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Loại bỏ dữ liệu trùng lặp

In [32]:
# ==============================================================
# Loại bỏ dữ liệu trùng lặp
# ==============================================================

before_count = df.count()
df = df.dropDuplicates(["CleanText"])
after_count = df.count()

print(f"Đã loại bỏ {before_count - after_count} dòng trùng lặp.")
print(f"Còn lại {after_count} dòng dữ liệu sau khi làm sạch & unique.")
df.select("CleanText").show(5, truncate=False)

Đã loại bỏ 81 dòng trùng lặp.
Còn lại 20967 dòng dữ liệu sau khi làm sạch & unique.


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CleanText                                                                                                                                                                                                                                                                                                                                                              |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Giữ cột quan trọng và lưu dữ liệu

In [33]:
# ==============================================================
# 🔹 Giữ cột cần thiết và lưu dữ liệu
# ==============================================================

keep_cols = ["Reviewer Name", "Country", "Review Date", "Rating", "CleanText", "Sentiment"]
df = df.select([c for c in keep_cols if c in df.columns])

print("Dữ liệu đã được xử lý và tinh gọn:")
df.show(5, truncate=False)

output_path = "../data/processed"
df.write.mode("overwrite").parquet(output_path)
print(f"Đã lưu dữ liệu tiền xử lý tại: {output_path}")


Dữ liệu đã được xử lý và tinh gọn:


+-------------+-------+-----------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|Reviewer Name|Country|Review Date|Rating|CleanText                                                                                                                                                                                                                                  

Đã lưu dữ liệu tiền xử lý tại: ../data/processed


### Data Validation sau khi preprocessing

In [34]:
# ==============================================================
# 🔹 Data Validation sau khi preprocessing
# ==============================================================

from pyspark.sql import functions as F

# Kiểm tra null
print("Kiểm tra null:")
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show(vertical=True)

# Kiểm tra trùng lặp
dup_count = df.groupBy("CleanText").count().filter(F.col("count") > 1).count()
print(f"Số dòng trùng lặp: {dup_count}")

# Kiểm tra phân bố rating
df.groupBy("Rating").count().orderBy("Rating").show()

print("Validation hoàn tất – dữ liệu đã sẵn sàng cho feature engineering.")


Kiểm tra null:


-RECORD 0------------
 Reviewer Name | 0   
 Country       | 0   
 Review Date   | 0   
 Rating        | 0   
 CleanText     | 0   
 Sentiment     | 0   



Số dòng trùng lặp: 0


+------+-----+
|Rating|count|
+------+-----+
|     1|13105|
|     2| 1225|
|     3|  879|
|     4| 1274|
|     5| 4484|
+------+-----+

Validation hoàn tất – dữ liệu đã sẵn sàng cho feature engineering.


## Xây dựng mô hình Machine Learning

### Import thư viện

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

### Đọc dữ liệu và trích xuất đặc trưng

In [36]:
df = spark.read.parquet("../data/processed/")

# Chuyển sang pandas để train model sklearn
pandas_df = df.select("CleanText", "Sentiment").toPandas()
pandas_df = pandas_df.dropna()

print("Dữ liệu sẵn sàng để train:", pandas_df.shape)
pandas_df.head()

Dữ liệu sẵn sàng để train: (20967, 2)


,CleanText,Sentiment
0,0 win complaint 0 win complaint amazon right ...,negative
1,100 shipping didn tell 100 shipping didn tell...,negative
2,10000000000 stars bad company 10000000000 sta...,negative
3,514 billion revenues least come good customer...,negative
4,600 order missing ve lot amazon orders incorr...,negative


### Chuẩn bị dữ liệu cho mô hình

In [38]:
X = pandas_df["CleanText"]
y = pandas_df["Sentiment"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF shape:", X_train_tfidf.shape)

TF-IDF shape: (16773, 5000)


### Định nghĩa 3 mô hình và tham số để Cross Validation

In [39]:
models = {
    "logistic_regression": {
        "model": LogisticRegression(max_iter=200),
        "params": {"C": [0.1, 1, 5]}
    },
    "svm": {
        "model": LinearSVC(),
        "params": {"C": [0.1, 1, 5]}
    },
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {"n_estimators": [100, 200], "max_depth": [10, 20]}
    }
}

### Train và Cross Validation cho từng mô hình

In [40]:
best_models = {}

for name, cfg in models.items():
    print(f"\n Training {name} ...")
    
    grid = GridSearchCV(
        estimator=cfg["model"],
        param_grid=cfg["params"],
        cv=5,
        scoring="accuracy",
        n_jobs=-1,
        verbose=1
    )
    
    grid.fit(X_train_tfidf, y_train)
    
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test_tfidf)
    acc = accuracy_score(y_test, y_pred)
    
    print(f" {name} accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Lưu model tốt nhất
    joblib.dump(best_model, f"../models/{name}_model.pkl")
    best_models[name] = (best_model, acc)


 Training logistic_regression ...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
 logistic_regression accuracy: 0.9266
              precision    recall  f1-score   support

    negative       0.93      0.97      0.95      2866
    positive       0.93      0.83      0.88      1328

    accuracy                           0.93      4194
   macro avg       0.93      0.90      0.91      4194
weighted avg       0.93      0.93      0.93      4194


 Training svm ...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
 svm accuracy: 0.9220
              precision    recall  f1-score   support

    negative       0.91      0.98      0.94      2866
    positive       0.95      0.80      0.87      1328

    accuracy                           0.92      4194
   macro avg       0.93      0.89      0.91      4194
weighted avg       0.92      0.92      0.92      4194


 Training random_forest ...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
 random_forest accuracy

In [41]:
best_name = max(best_models, key=lambda x: best_models[x][1])
print(f"\nMô hình tốt nhất: {best_name} (Accuracy = {best_models[best_name][1]:.4f})")


Mô hình tốt nhất: logistic_regression (Accuracy = 0.9266)
